In [1]:
import tensorflow as tf
import numpy as np 

In [2]:
def get_num_params():
    total_parameters = 0
    for variable in tf.trainable_variables():
        shape = variable.get_shape()
        # print(shape)
        # print(len(shape))
        variable_parameters = 1
        for dim in shape:
            # print(dim)
            variable_parameters *= dim.value
        # print(variable_parameters)
        total_parameters += variable_parameters
    return total_parameters



In [57]:
def ResentV2_block(input_data,input_depth,compress_depth,output_depth,strides=(1,1)):
    X_shortcut=input_data
    X=tf.layers.conv2d(input_data,compress_depth,(1,1))
    X=tf.layers.batch_normalization(X)
    X=tf.nn.leaky_relu(X)
    X=tf.layers.conv2d(X,compress_depth,(3,3),padding='same',strides=strides)
    X=tf.layers.batch_normalization(X)
    X=tf.nn.leaky_relu(X)
    X=tf.layers.conv2d(X,output_depth,(1,1))
    if (input_depth !=output_depth):
        X_shortcut=tf.layers.conv2d(X_shortcut,output_depth,(1,1),strides=strides,padding='same')
    if (input_depth ==output_depth) & (strides !=(1,1)):
        X_shortcut=tf.image.resize_images(X_shortcut, (X.shape[1], X.shape[2]), method=0)
    out=X_shortcut+X
    return out

In [60]:
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, [None, 64, 64, 128])
out=ResentV2_block(inputs,128,64,256,strides=(2,2))
print(out)

Tensor("add:0", shape=(?, 32, 32, 256), dtype=float32)


In [61]:
def Residual_Block(input_data, in_channel,out_channel,expansion = 4, s = 1):
    X_shortcut = input_data##記住輸入
    X = tf.layers.conv2d(input_data,out_channel, (1, 1), strides = (s,s))
    X = tf.layers.batch_normalization(X)
    X = tf.nn.relu(X)

    X =  tf.layers.conv2d(X,out_channel, (3, 3),padding='same', strides = (s,s))
    X = tf.layers.batch_normalization(X)
    X =tf.nn.relu(X)

    X =  tf.layers.conv2d(X,out_channel*expansion, (1,1), strides = (1, 1), )
    X =  tf.layers.batch_normalization(X)
    
    if (in_channel != out_channel*expansion) or (s != 1):
        X_shortcut=tf.layers.conv2d(X_shortcut,out_channel*expansion, (1,1), strides = (s, s))
        X_shortcut = tf.layers.batch_normalization(X)

    X = X+X_shortcut
    X = tf.nn.relu(X)
    return X

In [ ]:
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, [None, 64, 64, 128])
out=Residual_Block(inputs, 128,256,expansion = 4, s = 1)
print(out)

---

## Inception

In [5]:
import tensorflow as tf
def Inception(input_data,input_depth=192):
    with tf.name_scope('Branch_1'):
        X_1=tf.layers.conv2d(input_data,64,(1,1))
        X_1=tf.layers.batch_normalization(X_1)
        X_1=tf.nn.leaky_relu(X_1)
        
    with tf.name_scope('Branch_2'): 
        X_2=tf.layers.conv2d(input_data,96,(1,1))
        X_2=tf.layers.batch_normalization(X_2)
        X_2=tf.nn.leaky_relu(X_2)
        
        X_2=tf.layers.conv2d(X_2,128,(3,3),padding='same')
        X_2=tf.layers.batch_normalization(X_2)
        X_2=tf.nn.leaky_relu(X_2)
        
    with tf.name_scope('Branch_3'): 
        X_3=tf.layers.conv2d(input_data,16,(1,1))
        X_3=tf.layers.batch_normalization(X_3)
        X_3=tf.nn.leaky_relu(X_3)
        
        X_3=tf.layers.conv2d(X_3,48,(3,3),padding='same')
        X_3=tf.layers.batch_normalization(X_3)
        X_3=tf.nn.leaky_relu(X_3)

        X_3=tf.layers.conv2d(X_3,32,(5,5),padding='same')
        X_3=tf.layers.batch_normalization(X_3)
        X_3=tf.nn.leaky_relu(X_3)

    with tf.name_scope('Branch_4'): 
        X_4=tf.layers.max_pooling2d(input_data,2,1,padding='same')
        X_4=tf.layers.batch_normalization(X_4)
        X_4=tf.nn.leaky_relu(X_4)
        
        X_4=tf.layers.conv2d(X_4,32,(1,1),padding='same')
        X_4=tf.layers.batch_normalization(X_3)
        X_4=tf.nn.leaky_relu(X_3)

    out=tf.concat((X_1,X_2,X_3,X_4),axis=3)


    return out

In [6]:
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, [None, 28, 28, 192])
out=Inception(inputs,input_depth=192)
print(out)

Tensor("concat:0", shape=(?, 28, 28, 256), dtype=float32)


---

## Inception-Resnet

In [73]:
def InceptionResentA_block(input_data,input_depth=3,output_depth=384):
    X_shortcut=input_data
    with tf.name_scope('Branch_1'):
        X_1=tf.layers.conv2d(input_data,32,(1,1))
        X_1=tf.layers.batch_normalization(X_1)
        X_1=tf.nn.leaky_relu(X_1)
        
    with tf.name_scope('Branch_2'): 
        X_2=tf.layers.conv2d(input_data,32,(1,1))
        X_2=tf.layers.batch_normalization(X_2)
        X_2=tf.nn.leaky_relu(X_2)
        
        X_2=tf.layers.conv2d(X_2,32,(3,3),padding='same')
        X_2=tf.layers.batch_normalization(X_2)
        X_2=tf.nn.leaky_relu(X_2)
        
    with tf.name_scope('Branch_3'): 
        X_3=tf.layers.conv2d(input_data,32,(1,1))
        X_3=tf.layers.batch_normalization(X_3)
        X_3=tf.nn.leaky_relu(X_3)
        
        X_3=tf.layers.conv2d(X_3,48,(3,3),padding='same')
        X_3=tf.layers.batch_normalization(X_3)
        X_3=tf.nn.leaky_relu(X_3)
        
        
        X_3=tf.layers.conv2d(X_3,64,(3,3),padding='same')
        X_3=tf.layers.batch_normalization(X_3)
        X_3=tf.nn.leaky_relu(X_3)
    out=tf.concat((X_1,X_2,X_3),axis=3)

    out=tf.layers.conv2d(out,output_depth,(1,1))

    if (input_depth !=output_depth):
        X_shortcut=tf.layers.conv2d(X_shortcut,output_depth,(1,1))

    out=X_shortcut+out
    return out

In [74]:
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, [None, 64, 64, 3])
out=InceptionResentA_block(inputs,input_depth=3,output_depth=384)
print(out)

Tensor("add:0", shape=(?, 64, 64, 384), dtype=float32)


In [77]:

def Dense_Stage(inputs_,depth=64,repeat=8):
    for _ in range(repeat):
        X_input=inputs_
        X=tf.layers.conv2d(inputs_,depth,(1,1),strides=(1,1),activation=tf.nn.leaky_relu)
        X=tf.layers.batch_normalization(X)
        X=tf.layers.separable_conv2d(X,depth,(3,3),padding='SAME')
        X=tf.nn.leaky_relu(X)
        X=tf.layers.batch_normalization(X)
        X=tf.concat([X_input,X],3)
        inputs_=X
    return X

In [79]:
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, [None, 64, 64, 3])
out=Dense_Stage(inputs,depth=64,repeat=8)
print(out)

Tensor("concat_7:0", shape=(?, 64, 64, 515), dtype=float32)


---

## Depthwise


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [9]:
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, [None, 300, 300, 3])
out=depthwise_conv(
        inputs, kernel=3, stride=1, padding='SAME',
        activation_fn=None, normalizer_fn=None,
        weights_initializer=tf.contrib.layers.xavier_initializer(),
        data_format='NHWC', scope='depthwise_conv')
out=tf.layers.conv2d(out,64,(1,1))
print(get_num_params()) 

283


In [ ]:
import tensorflow as tf
import tensorflow.contrib as tc

slim = tc.slim
def depthwise_conv_bn(x, kernel_size, stride=1, dilation=1):
    with tf.variable_scope(None, 'depthwise_conv_bn'):
        x = slim.separable_conv2d(x, None, kernel_size, depth_multiplier=1, stride=stride,
                                  rate=dilation, activation_fn=None, biases_initializer=None)
        #x = slim.batch_norm(x, activation_fn=None, fused=False)
    return x
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, [None, 300, 300, 3])
out=depthwise_conv_bn(inputs, (3,3), stride=1, dilation=1)
print(get_num_params()) 

---

## shufflenetV2

In [118]:
##參考：https://github.com/timctho/shufflenet-v2-tensorflow/blob/master/module.py
##參考：https://github.com/TropComplique/shufflenet-v2-tensorflow/blob/master/architecture.py

def shuffle_unit(x, groups):  ##一般的shuffle depthwise_conv輸出的Feature Map
    with tf.variable_scope('shuffle_unit'):
        n, h, w, c = x.get_shape().as_list()
        x = tf.reshape(x, shape=([tf.shape(x)[0], h, w, groups, c // groups]))
        x = tf.transpose(x, tf.convert_to_tensor([0, 1, 2, 4, 3]))
        x = tf.reshape(x, shape=[tf.shape(x)[0], h, w, c])
    return x
def depthwise_conv(
        x, kernel=3, stride=1, padding='SAME',
        activation_fn=None, normalizer_fn=None,
        weights_initializer=tf.contrib.layers.xavier_initializer(),
        data_format='NHWC', scope='depthwise_conv'):      ##一般的depthwise_conv

    with tf.variable_scope(scope):
        assert data_format == 'NHWC'
        in_channels = x.shape[3].value
        W = tf.get_variable(
            'depthwise_weights',
            [kernel, kernel, in_channels, 1], dtype=tf.float32,
            initializer=weights_initializer
        )
        x = tf.nn.depthwise_conv2d(x, W, [1, stride, stride, 1], padding, data_format='NHWC')
        x = tf.layers.batch_normalization(x) if normalizer_fn is not None else x  # batch normalization
        x = tf.nn.leaky_relu(x) if activation_fn is not None else x  # nonlinearity
        return x
    
def conv_bn_relu(x, out_channel, kernel_size, stride=1):  ##一般的Convolution+BN+Relu
    with tf.variable_scope(None, 'conv_bn_relu'):
        x = tf.layers.conv2d(x, out_channel, kernel_size, stride,)
        x = tf.nn.leaky_relu(tf.layers.batch_normalization(x))
    return x

def shufflenet_v2_block(x, out_channel, kernel_size, stride=1, shuffle_group=2): ##shufflenet_v2_block
    with tf.variable_scope(None, 'shuffle_v2_block'):
        if stride == 1:
            top, bottom = tf.split(x, num_or_size_splits=2, axis=3)

            half_channel = out_channel // 2

            top = conv_bn_relu(top, half_channel, 1)
            top = depthwise_conv_bn(top, kernel_size, stride)
            top = conv_bn_relu(top, half_channel, 1)

            out = tf.concat([top, bottom], axis=3)
            out = shuffle_unit(out, shuffle_group)

        else:   ##downsampling的Block
            half_channel = out_channel // 2
            b0 = conv_bn_relu(x, half_channel, 1)
            b0 = depthwise_conv_bn(b0, kernel_size, stride)
            b0 = conv_bn_relu(b0, half_channel, 1)

            b1 = depthwise_conv_bn(x, kernel_size, stride)
            b1 = conv_bn_relu(b1, half_channel, 1)

            out = tf.concat([b0, b1], axis=3)
            out = shuffle_unit(out, shuffle_group)
        return out

tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, [None, 300, 300, 4])
out=shufflenet_v2_block(inputs, 2, (3,3), stride=1, shuffle_group=2)
print(get_num_params()) 

18


## tf.variable_scope

In [103]:
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, [None, 300, 300, 6])
with tf.variable_scope(None, 'depthwise_conv_bn'):
    x=tf.layers.conv2d(inputs,10,(1,1),strides=(1,1),activation=tf.nn.leaky_relu)
    print(x)
with tf.variable_scope(None, 'depthwise_conv_bn'):
    x=tf.layers.conv2d(x,10,(1,1),strides=(1,1),activation=tf.nn.leaky_relu)
    print(x)

Tensor("depthwise_conv_bn/conv2d/LeakyRelu:0", shape=(?, 300, 300, 10), dtype=float32)
Tensor("depthwise_conv_bn_1/conv2d/LeakyRelu:0", shape=(?, 300, 300, 10), dtype=float32)


---

## MobilenetV2

In [129]:
def depthwise_conv(
        x, kernel=3, stride=1, padding='SAME',
        activation_fn=None, normalizer_fn=None,
        weights_initializer=tf.contrib.layers.xavier_initializer(),
        data_format='NHWC', scope='depthwise_conv'):      ##一般的depthwise_conv

    with tf.variable_scope(scope):
        assert data_format == 'NHWC'
        in_channels = x.shape[3].value
        W = tf.get_variable(
            'depthwise_weights',
            [kernel, kernel, in_channels, 1], dtype=tf.float32,
            initializer=weights_initializer
        )
        x = tf.nn.depthwise_conv2d(x, W, [1, stride, stride, 1], padding, data_format='NHWC')
        x = tf.layers.batch_normalization(x) if normalizer_fn is not None else x  # batch normalization
        x = tf.nn.leaky_relu(x) if activation_fn is not None else x  # nonlinearity
        return x


def res_block(input, expansion_ratio, output_dim, stride, name, bias=False, shortcut=True):
    with tf.name_scope(name), tf.variable_scope(name):
        # pw
        bottleneck_dim=round(expansion_ratio*input.get_shape().as_list()[-1]) 
        net = tf.layers.conv2d(input, bottleneck_dim,(1,1), name='pw', 
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(0.003),use_bias=bias) ##先擴張
        net = tf.layers.batch_normalization(net, name='pw_bn')
        net = tf.nn.relu6(net)
        # dw
        net = depthwise_conv(net)
        net = tf.layers.batch_normalization(net, name='dw_bn')
        net = tf.nn.relu6(net)
        # pw & linear
        net = tf.layers.conv2d(net, output_dim,(1,1), name='pw_linear', 
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(0.003),use_bias=bias) ##壓回輸出深度
        net = tf.layers.batch_normalization(net,name='pw_linear_bn')

        # element wise add, only for stride==1
        if shortcut and stride == 1:
            in_dim=int(input.get_shape().as_list()[-1])
            if in_dim != output_dim:
                ins=tf.layers.conv2d(input, output_dim,(1,1), name='ex_dim', 
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(0.003),use_bias=bias) 
                net=ins+net
            else:
                net=input+net

        return net




In [130]:
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, [None, 300, 300, 3])
out=res_block(inputs,4,32,1,name='Res1')
out=res_block(out,4,32,1,name='Res2')

---

## MBConvBlock

In [35]:
import tensorflow as tf
def depthwise_conv(
        x, kernel=3, stride=1, padding='SAME',
        activation_fn=None, normalizer_fn=None,
        weights_initializer=tf.contrib.layers.xavier_initializer(),
        data_format='NHWC', scope='depthwise_conv'):      ##一般的depthwise_conv

    with tf.variable_scope(scope):
        assert data_format == 'NHWC'
        in_channels = x.shape[3].value
        W = tf.get_variable(
            'depthwise_weights',
            [kernel, kernel, in_channels, 1], dtype=tf.float32,
            initializer=weights_initializer
        )
        x = tf.nn.depthwise_conv2d(x, W, [1, stride, stride, 1], padding, data_format='NHWC')
        x = tf.layers.batch_normalization(x) if normalizer_fn is not None else x  # batch normalization
        x = tf.nn.leaky_relu(x) if activation_fn is not None else x  # nonlinearity
        return x


def MBConvBlock(input, expansion_ratio, output_dim, stride, name,squeeze ,bias=False, shortcut=True,use_Squeeze_Excitation=True):
    with tf.name_scope(name), tf.variable_scope(name):
        # pw
        bottleneck_dim=round(expansion_ratio*input.get_shape().as_list()[-1]) 
        net = tf.layers.conv2d(input, bottleneck_dim,(1,1), name='pw', 
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(0.003),use_bias=bias) ##先擴張
        net = tf.layers.batch_normalization(net, name='pw_bn')
        net = tf.nn.relu6(net)
        # dw
        net = depthwise_conv(net,stride=stride)
        net = tf.layers.batch_normalization(net, name='dw_bn')
        net = tf.nn.relu6(net)
        # pw & linear
        net = tf.layers.conv2d(net, output_dim,(1,1),name='pw_linear', 
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(0.003),use_bias=bias) ##壓回輸出深度
        net = tf.layers.batch_normalization(net,name='pw_linear_bn')
        
        if use_Squeeze_Excitation:
            in_dim=int(net.get_shape().as_list()[-1])
            Squeeze=tf.layers.average_pooling2d(net, net.get_shape()[1:-1], 1)
            Squeeze=tf.nn.relu(tf.layers.dense(Squeeze, use_bias=False, units=in_dim//squeeze))
            Excitation=tf.nn.relu(tf.layers.dense(Squeeze, use_bias=False, units=output_dim))
            Excitation=tf.nn.sigmoid(Excitation)
            net = tf.reshape(Excitation, [-1,1,1,output_dim])*net

        # SENET-Squeeze-Excitation
        in_dim=int(input.get_shape().as_list()[-1])
        if shortcut and stride == 1:
            if in_dim != output_dim:
                ins=tf.layers.conv2d(input, output_dim,(1,1), name='ex_dim', 
                               kernel_regularizer=tf.contrib.layers.l2_regularizer(0.003),use_bias=bias) 
                net=ins+net
            else:
                net=input+net

        return net
    
    
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, [None, 300, 300, 43])
out=MBConvBlock(inputs, 4, 64, 1, 'first', 4,bias=False, shortcut=True,use_Squeeze_Excitation=True)



In [36]:
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, [None, 300, 300, 43])
out=MBConvBlock(inputs, 4, 64, 1, 'first', 4,bias=False, shortcut=True,use_Squeeze_Excitation=True)


---

## MobilenetV3

In [2]:
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [59]:
with tf.Session() as sess:
    print(sess.run(tf.nn.relu6(-4 + 3.) / 6.))

0.0


In [10]:
import tensorflow as tf
def Hswish(input_):
    return input_* tf.nn.relu6(input_ + 3.) / 6.

def Hsigmoid(input_):
    return tf.nn.relu6(input_ + 3.) / 6.

def depthwise_conv(
        x, kernel=3, stride=1, padding='SAME',
        activation_fn=None, normalizer_fn=None,
        weights_initializer=tf.contrib.layers.xavier_initializer(),
        data_format='NHWC', scope='depthwise_conv'):      ##一般的depthwise_conv

    with tf.variable_scope(scope):
        assert data_format == 'NHWC'
        in_channels = x.shape[3].value
        W = tf.get_variable(
            'depthwise_weights',
            [kernel, kernel, in_channels, 1], dtype=tf.float32,
            initializer=weights_initializer
        )
        x = tf.nn.depthwise_conv2d(x, W, [1, stride, stride, 1], padding, data_format='NHWC',)
        x = tf.layers.batch_normalization(x) if normalizer_fn is not None else x  # batch normalization
        x = tf.nn.leaky_relu(x) if activation_fn is not None else x  # nonlinearity
        return x
    
def SEBlock(input_,squeeze=4):
    in_dim=int(input_.get_shape().as_list()[-1])
    print( input_.get_shape()[1:-1])
    Squeeze=tf.layers.average_pooling2d(input_, input_.get_shape()[1:-1], 1)
    Squeeze=tf.nn.relu(tf.layers.dense(Squeeze, use_bias=False, units=in_dim//squeeze)) 
    Excitation=tf.nn.relu(tf.layers.dense(Squeeze, use_bias=False, units=in_dim))
    Excitation=Hsigmoid(Excitation) ##Hsigmoid replace Sigmoid
    Excitation = tf.reshape(Excitation, [-1,1,1,in_dim])
    return input_*Excitation
    
    
def MobileV3Bottleneck(input_,expand_size,squeeze,out_size,kernel_size,stride=1,relu=True,se=True):
    Shortcut=input_
    in_dim=int(input_.get_shape().as_list()[-1])
    out=tf.layers.batch_normalization(tf.layers.conv2d(input_,expand_size,(1,1),(1,1),use_bias=False))
    if relu:
        out=tf.nn.relu(out) #or relu6
    else:
        out=Hswish(out)
        
    
    out=depthwise_conv(out,kernel=kernel_size,stride=stride, padding='SAME')
    out=tf.layers.batch_normalization(out)
    if relu:
        out=tf.nn.relu(out) #or relu6
    else:
        out=Hswish(out)
        
    out=tf.layers.batch_normalization(tf.layers.conv2d(input_,out_size,(1,1),(1,1),use_bias=False))
    
    if (in_dim != out_size) and (stride == 1):
        Shortcut=tf.layers.conv2d(Shortcut,out_size, (1,1), strides = (stride, stride),use_bias=False)
        Shortcut = tf.layers.batch_normalization(Shortcut)
    if se:
        assert squeeze<=out_size
        out=SEBlock(out,squeeze=squeeze)

    out = out +Shortcut if stride==1 else out
    return out
    



In [11]:

    
    
tf.reset_default_graph()
inputs = tf.placeholder(tf.float32, [None, 300, 300, 80])
out=MobileV3Bottleneck(inputs,480,4,112,3,stride=1,relu=False,se=True)
  
    

(300, 300)
